In [1]:
#import
import os

#for operations
import pandas as pd
import numpy as np

#for AWS connection
from boto.s3.connection import S3Connection
from IPython.parallel import Client

In [2]:
#get creds for aws
credentials = pd.read_csv('/home/centos/certificates/credentials.csv')
aws_id = credentials['Access Key Id'][0]
aws_key = credentials['Secret Access Key'][0]

In [3]:
#connect to S3 for data
s3conn = S3Connection( aws_id , aws_key )
bucket = s3conn.get_bucket('avs-kaggle-data')

### Load offers data

In [4]:
offers_keys = bucket.get_all_keys(prefix='offers')
len( offers_keys)

1

In [5]:
offers_keys[0].get_contents_to_filename('offers.csv')

In [6]:
offers_df = pd.read_csv('offers.csv')
print offers_df.shape
offers_df.head(3)

(37, 6)


,offer,category,quantity,company,offervalue,brand
0,1190530,9115,1,108500080,5.00,93904
1,1194044,9909,1,107127979,1.00,6732
2,1197502,3203,1,106414464,0.75,13474


### Load test_hist data

In [7]:
train_hist_keys = bucket.get_all_keys(prefix='trainHistory')
len( train_hist_keys)

1

In [8]:
train_hist_keys[0].get_contents_to_filename('trainHistory.csv')

In [9]:
train_hist_df = pd.read_csv('trainHistory.csv')
print train_hist_df.shape
train_hist_df.head(3)

(160057, 7)


,id,chain,offer,market,repeattrips,repeater,offerdate
0,86246,205,1208251,34,5,t,2013-04-24
1,86252,205,1197502,34,16,t,2013-03-27
2,12682470,18,1197502,11,0,f,2013-03-28


### merge offers and test_hist data

In [10]:
train_hist_offers_df = pd.merge( train_hist_df, offers_df, how='inner')

print train_hist_offers_df.shape
train_hist_offers_df.head(5)

(160057, 12)


,id,chain,offer,market,repeattrips,repeater,offerdate,category,quantity,company,offervalue,brand
0,86246,205,1208251,34,5,t,2013-04-24,2202,1,104460040,2,3718
1,15753725,17,1208251,4,0,f,2013-04-24,2202,1,104460040,2,3718
2,16535563,4,1208251,1,7,t,2013-04-27,2202,1,104460040,2,3718
3,18259179,3,1208251,2,0,f,2013-04-24,2202,1,104460040,2,3718
4,21024070,15,1208251,9,1,t,2013-04-23,2202,1,104460040,2,3718


### import *relevant* transaction data in chunk_rows

In [11]:
trans_keys = bucket.get_all_keys(prefix='trans')
len( trans_keys)
print trans_keys

[<Key: avs-kaggle-data,transactions-sample.csv>, <Key: avs-kaggle-data,transactions.csv.gz>, <Key: avs-kaggle-data,transactions.gz>]


In [22]:
trans_keys[1].get_contents_to_filename('transactions.csv.gz')

In [23]:
#test w/o 'compression' kwarg ==> works
col_df = pd.read_csv( 'transactions.csv.gz',  
                     compression='gzip',
                     nrows=1) 

col_headers = list( col_df.columns)
print col_headers

col_df.head()

['id', 'chain', 'dept', 'category', 'company', 'brand', 'date', 'productsize', 'productmeasure', 'purchasequantity', 'purchaseamount']


,id,chain,dept,category,company,brand,date,productsize,productmeasure,purchasequantity,purchaseamount
0,86246,205,7,707,1078778070,12564,2012-03-02,12,OZ,1,7.59


In [24]:
col_headers = list(col_df.columns)
col_headers

['id',
 'chain',
 'dept',
 'category',
 'company',
 'brand',
 'date',
 'productsize',
 'productmeasure',
 'purchasequantity',
 'purchaseamount']

In [25]:
#define variables
n=0 #test variable
chunk_rows = 1000000
rows_hist_train = 5000000 #test
#rows_hist_train = 349655790 #actual figure, from bash cmd

#iterate through transactions
for skip_rows in range( 0, rows_hist_train, chunk_rows):
    print skip_rows

0
1000000
2000000
3000000
4000000


In [26]:
import pandas as pd
import gzip

In [28]:
#create empty df to append to
train_df = pd.DataFrame()

#get headers
#col_df = pd.read_csv( 'transactions.csv.gz', 
#                     #compression='gzip', #define for local, NOT for EC2
#                     engine='python',
#                     nrows=1)
#col_headers = list(col_df.columns)
#print col_headers[4]
#if isinstance (col_headers, list):
#    print "ok list"

#define variables
#n=0 #test variable
chunk_rows = 100000
rows_hist_train = 500000 #test
#rows_hist_train = 349655790 #actual figure, from bash cmd

#do first w/ headers NOT specificied
#temp_df = pd.read_csv('transactions.csv.gz',nrows= chunk_rows) 
#temp_df.columns = col_headers
#temp_df_2 = pd.merge( temp_df, train_hist_offers_df, how='inner')
#train_df = train_df.append( temp_df_2)

#print train_df.shape

#iterate through transactions
skip_rows=0
print type (skip_rows) is int

#for skip_rows in range( 0, rows_hist_train, chunk_rows): #start second loop from end of first loop
    
#test function inputs
#print("skip_rows: %7d, chunk_rows: %7d" %(skip_rows, chunk_rows)) 
#print ("n: %1d" %n)

#read
temp_df2 = pd.DataFrame()

temp_df2 = pd.read_csv("transactions.csv.gz", compression='gzip',engine='python',skiprows = 400000, nrows= 100000)
temp_df2.head()    
#                      names='col_headers') 
                      #header='col_headers',
                      #compression= 'gzip', #define for local, NOT for EC2
                      #compression='bz2', #ramesh suggestion
                      #engine='python')
                      #skiprows= skip_rows)
#n = n+1 #increment for test

#merge with train_hist_offers to pull ONLY relevant data, then append
#temp_df.columns = col_headers
#temp_df_2 = pd.merge( temp_df, train_hist_offers_df, how='inner')
#train_df = train_df.append( temp_df_2)

#write to csv
#train_df.to_csv('train_df_test.csv')


True


,61599063,15,63,6332,104850040,61745,2013-02-23,59,OZ,1,3
0,61599063,15,59,5902,105820050,10786,2013-02-23,16.0,OZ,1,3.29
1,61599063,15,25,2509,102113020,10786,2013-02-23,16.0,OZ,1,1.59
2,61599063,15,9,914,107343575,9705,2013-02-26,4.4,OZ,1,1.00
3,61599063,15,36,3611,102113020,15704,2013-02-26,33.8,OZ,2,1.78
4,61599063,15,10,1011,104300040,26232,2013-02-26,6.7,OZ,1,1.99


In [16]:
print n

0
